# Data Mining And Machine Learning For IOT Applications In Industry at SAIL, Bokaro, India - Linear Regression

The contents of this notebook are taken (and modified) from the COMP90051 course (workshop) at the University of Melbourne, Australia.



## Part A: Simple Linear regression

***

The aim of this part of the workshop is to get you coding linear regression models in Python, relying on the `numpy` library. We will do it in three ways: One based on approximate iterative updates (coordinate descenet), second based on linear algebra (Normal equations), and third based on `sklearn`. The correctness of the first two implementations (from scratch) will be verified by comparing their ouputs to the output of `sklearn`.


Firstly, we will import the relevant libraries (`numpy`, `matplotlib`, etc.).

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import io

To check what a command does simply type `object?`. For example:

In [ ]:
np.random.randn?

### 1. Review
In the lecture, we saw that a linear model can be expressed as:
$$y = w_0 + \sum_{j = 1}^{d} w_j x_j = \mathbf{w} \cdot \mathbf{x} $$
where 

* $y$ is the *response or target variable*;
* $\mathbf{x} = [x_1, \ldots, x_d]$ is a vector of *features* (we define $x_0 = 1$); and
* $\mathbf{w} = [w_0, \ldots, w_d]$ are the *weights*.

To fit the model, we *minimise* the sum of squared residuals (errors), SSR:

$$SSR(\mathbf{w}) = \sum_{i=1}^{n}(y^{(i)} - \mathbf{w} \cdot \mathbf{x}^{(i)})^2$$

**Note:** For simplicity, we'll consider the case $d = 1$ (i.e. only one feature excluding the intercept).

### 2. Data set
We'll be working with some data from the Olympics—the gold medal race times for marathon winners from 1896 to 2012. The code block below reads the data into a numpy array of floats, and prints the result.

In [ ]:
# CSV file with variables YEAR,TIME
csv = """1896,4.47083333333333
1900,4.46472925981123
1904,5.22208333333333
1908,4.1546786744085
1912,3.90331674958541
1920,3.5695126705653
1924,3.8245447722874
1928,3.62483706600308
1932,3.59284275388079
1936,3.53880791562981
1948,3.6701030927835
1952,3.39029110874116
1956,3.43642611683849
1960,3.2058300746534
1964,3.13275664573212
1968,3.32819844373346
1972,3.13583757949204
1976,3.07895880238575
1980,3.10581822490816
1984,3.06552909112454
1988,3.09357348817
1992,3.16111703598373
1996,3.14255243512264
2000,3.08527866650867
2004,3.1026582928467
2008,2.99877552632618
2012,3.03392977050993"""

# Read into a numpy array (as floats)
olympics = np.genfromtxt(io.BytesIO(csv.encode()), delimiter=",")
print(olympics)

This loads the data into numpy array. We'll take the race time as the *target variable* $y$ and the year of the race as the only *feature* $x = x_1$.

In [ ]:
x = olympics[:, 0:1]
y = olympics[:, 1:2]

And you can make a plot of $y$ vs $x$ with the following commands. Can a linear model be a decent fit for this data?

In [ ]:
plt.plot(x, y, 'rx')
plt.ylabel("y (Race time)")
plt.xlabel("x (Year of race)")
plt.show()

### 3. Approximate Iterative solution (coordinate descent)

Expanding out the sum of square residuals for this simple case (where $\mathbf{w}=[w_0, w_1]$) we have:
$$SSR(w_0, w_1) = \sum_{i=1}^{n}(y_i - w_0 - w_1 x_i)^2$$
Let's start with an initial guess for the slope $w_1$ (which is clearly negative from the plot).

In [ ]:
w1 = -0.4

Then using the maximum likelihood update, we get the following estimate for the intercept $w_0$:
$$w_0 = \frac{\sum_{i=1}^{n}(y_i-w_1 x_i)}{n}$$

In [ ]:
w0 = ... # over to you
print(w0)

Similarly, we can update $w_1$ based on this new estimate of $w_0$:
$$w_1 = \frac{\sum_{i=1}^{n} (y_i - w_0) \times x_i}{\sum_{i=1}^{n} x_i^2}$$

In [ ]:
w1 = ... # over to you
print(w1)

Let's examine the quality of fit for these values for the weights $w_0$ and $w_1$. First, we create a vector of "test" values `x_test`.

In [ ]:
x_test = np.arange(1890, 2020)[:, None]

then, use this vector to compute some test predictions

In [ ]:
y_test = ... # over to you

Now plot the test predictions with a blue line on the same plot as the data.

In [ ]:
def plot_fit(x_test, y_test, x, y): 
    plt.plot(x_test, y_test, 'b-')
    plt.plot(x, y, 'rx')
    plt.ylabel("y (Race time)")
    plt.xlabel("x (Year of race)")
    plt.show()

plot_fit(x_test, y_test, x, y)

Next, we compute the quality of the fit by evaluating the average sum of squares error of the prediction over the training samples, $SSR(w_0,w_1)$

In [ ]:
def compute_SSR(x, y, w0, w1): 
    return ... # over to you

print(compute_SSR(x, y, w0, w1))

It's obvious from the plot that the fit isn't very good. 
We must repeat the alternating parameter updates many times before the algorithm converges to the optimal weights.

In [ ]:
for i in np.arange(10000):
    w1 = ... # paste from above
    w0 = ... # paste from above
    if i % 500 == 0:
        print("Iteration #{}: SSR = {}".format(i, compute_SSR(x, y, w0, w1)))
print("Final estimates: w0 = {}; w1 = {}".format(w0, w1))

Let's try plotting the result again.

In [ ]:
y_test = ... # paste from above
plot_fit(x_test, y_test, x, y)

### 4. Linear algebra solution

In lecture, we saw that it's possible to solve for the optimal weights $\mathbf{w}^\star$ analytically. The solution is
$$\mathbf{w}^* = \left[\mathbf{X}^\top \mathbf{X}\right]^{-1} \mathbf{X}^\top \mathbf{y}$$
where
$$\mathbf{X} = \begin{pmatrix} 
        1 & x_1 \\ 1 & x_2 \\ \vdots & \vdots \\ 1 & x_n 
    \end{pmatrix} 
  \quad \text{and} \quad 
  \mathbf{y} = \begin{pmatrix} 
          y_1 \\ y_2 \\ \vdots \\ y_n
      \end{pmatrix}
$$

We construct $\mathbf{X}$ in the code block below, remembering to include the $x_0 = 1$ column for the bias.

In [ ]:
X = np.hstack((np.ones_like(x), x))
print(X)

Although we can express $\mathbf{w}^\star$ explicitly in terms of the matrix inverse $(\mathbf{X}^\top \mathbf{X})^{-1}$, this isn't an efficient way to compute $\mathbf{w}$ numerically. It is better instead to solve the following system of linear equations:
$$\mathbf{X}^\top\mathbf{X} \mathbf{w}^\star = \mathbf{X}^\top\mathbf{y}$$

This can be done in numpy using the command

In [ ]:
np.linalg.solve?

which gives

In [ ]:
w = ... # back to you
print(w)

Plotting this solution, as before:

In [ ]:
w0, w1 = w
y_test = ... # paste from above 
plot_fit(x_test,y_test, x, y)

You should verify that the sum of squared residuals $SSR(w_0, w_1)$, match or beats the earlier iterative result.

In [ ]:
print(compute_SSR(x, y, w0, w1))

###  5. Solving using scikit-learn

Now that you have a good understanding of what's going on under the hood, you can use the functionality in `sklearn` to solve linear regression problems you encounter in the future. Using the `LinearRegression` module, fitting a linear regression model becomes a one-liner as shown below.

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(x, y)

The `LinearRegression` module provides access to the bias weight $w_0$ under the `intercept_` property

In [ ]:
lr.intercept_

and the non-bias weights under the `coef_` property

In [ ]:
lr.coef_

In [ ]:
y_test = lr.intercept_ + lr.coef_ * x_test
plot_fit(x_test,y_test, x, y)

In [ ]:
print(compute_SSR(x, y, w0, w1))

You should check that these results match the solution you obtained previously.

## Part B: Model Complexity and Regularization

This part will review overfitting, model selection and regularisation. Note that the lessons here apply equally to classification, however it's more convenient to visualise regression models, and they are also much simpler to fit to data.

**Discussion:** We will consider regression models of varying complexity, from a simple linear model to polynomial models of varying order.
Q1: Based on the Olympic marathon data, what order model do you think is going to perform the best? (*Hint*: In making your decision, think about the *interpolation* predictions for years between Olympics (e.g., 2015), and *extrapolations* into the future, e.g., 2016, 2020, 2040, etc?

## Polynomial Regression

Now we will consider a more complex polynomial function. Where before we had instances of the form,
$$\phi(\mathbf{x}) = [ 1~ x ]$$ 
now we will be using e.g., 
$$\phi(\mathbf{x}) = [ 1 ~x~ x^2~ x^3~ x^4]$$ 
for a quartic model. We will consider a range of polynomial models of different orders. 

To implement this we will use *basis functions* which provide a neat way of representing our data instances such that we can still use all the linear models to acheive learn a non-linear model. 

### Data Preparation

In [ ]:
num_data = x.shape[0]
num_pred_data = 100 # how many points to use for plotting predictions
x_pred = np.linspace(1890, 2016, num_pred_data)[:, None] # input locations for predictions
order = 4 # The polynomial order to use.
print ('Num of training samples: ',num_data)
print('Num of testing samples: ',num_pred_data)

Now let's build the *basis* matrices $\Phi$ to represent the training data, where each column is raising the input year $X$ to various powers.

In [ ]:
Phi = np.zeros((num_data, order+1))
Phi_pred = np.zeros((num_pred_data, order+1))
for i in range(0, order+1):
    Phi[:, i:i+1] = x**i
    Phi_pred[:, i:i+1] = x_pred**i

In [ ]:
Phi

### Fitting the model

Now we can solve for the regression weights and make predictions both for the training data points, and the test data points. That involves solving the linear system given by

$$\Phi' \Phi \mathbf{w} = \Phi' \mathbf{y}$$

with respect to $\mathbf{w}$.

In [ ]:
# solve the linear system
w = ... # over to you

In [ ]:
#use resulting vector to make predictions at the training points and test points
f = ... # over to you
f_pred = ... # over to you

In [ ]:
# compute the sum of squares error
SSR =  ... # paste from above    

In [ ]:
#Now we have the fit and the error, so let's plot the fit and the error.
print("The error is: %2.4f"%SSR)
plt.plot(x_pred, f_pred)
plt.plot(x, y, 'rx')
ax = plt.gca()
ax.set_title('Predictions for Order 5')
ax.set_xlabel('year')
ax.set_ylabel('pace (min/km)')

Now use the loop structure below to compute the error for different model orders.

In [ ]:
# import the time model to allow python to pause.
# import the IPython display module to clear the output.
import time
from IPython.display import clear_output

error_list = []
max_order = 6
#fig, axes = plt.subplots(nrows=1, ncols=2)
fig1=plt.figure(figsize=(15,2*max_order))
index=1

for order in range(0, max_order+1):
    # 1. build the basis set
    Phi = np.zeros((num_data, order+1))
    Phi_pred = np.zeros((num_pred_data, order+1))
    for i in range(0, order+1):
        Phi[:, i:i+1] = .. # paste from above
        Phi_pred[:, i:i+1] = .. # paste from above
    # 2. solve the linear system
    w = ... # paste from above

    # 3. make predictions at training and test points
    f = ... # paste from above
    f_pred = ... # paste from above
    
    # 4. compute the error and append it to a list.
    SSR = ... # paste from above    
    error_list.append(SSR)
    
    # 5. plot the predictions
    fig1.add_subplot(max_order+1,2,index)
    plt.plot(x_pred, f_pred)
    plt.plot(x, y, 'rx')
    plt.ylim((2.5, 5.5))
    if (order ==0):
        plt.title('Predictions for Order ' + str(order) + ' model.')
    
    
    fig1.add_subplot(max_order+1,2,index+1)
    plt.plot(np.arange(0, order+1), np.asarray(error_list))
    plt.xlim((0, order+1))
    plt.ylim((0, np.max(error_list)))
    if (order ==0):
        plt.title('Training Error')
    index= index+2

plt.show()
#display(fig)
print('Training error list: ',error_list)

**Discussion:** Looks like a great fit. Does that mean we can stop here, our job is done? You might want to try an order 20 or higher model, also to see if the fits continue to improve with higher order models.

**Discussion:** What do you think might happen if we try to fit an order 100 model to this data? Is this even a reasonable thing to try?

## Hold Out Validation

The error we computed above is the training error. It doesn't assess the model's generalization ability, it only assesses how well it's performing on the given training data. 


In hold out validation, we keep back some of the training data for assessing generalization performance. In the case of time series prediction, it often makes sense to hold out the last few data points, in particular, when we are interested in *extrapolation*, i.e. predicting into the future given the past. To perform hold out validation, we first remove the hold out set. If we were interested in interpolation, we would hold out some random points. Here, because we are interested in extrapolation, we will hold out all points since 1980. 

In [ ]:
# Create a training set
x_train = x
y_train = y
indices_hold_out = np.nonzero(x>1980)


x_train = np.delete(x, indices_hold_out)[:,None]
y_train = np.delete(y, indices_hold_out)[:,None]

# Create a hold out set
x_hold_out = x[indices_hold_out][:,None]
y_hold_out = y[indices_hold_out][:,None]


print ('Whole dataset size', x.shape)
print('Train split size: ', x_train.shape)
print('Test split size: ', x_hold_out.shape)

# Now use the training set and hold out set.

Now you have the training and hold out data, you should be able to use the code above to evaluate the model on the hold out data. Do this in the code block below.

In [ ]:
error_list = []
max_order = 6
#fig, axes = plt.subplots(nrows=1, ncols=2)
fig1=plt.figure(figsize=(12,max_order*2))
index = 1
for order in range(0, max_order+1):
    # 1. build the basis set using x_train, x_hold_out
    Phi = np.zeros((x_train.shape[0], order+1))
    Phi_pred = np.zeros((num_pred_data, order+1))
    Phi_hold_out = np.zeros((x_hold_out.shape[0], order+1))
    for i in range(0, order+1):
        Phi[:, i:i+1] = ... # back to you
        Phi_hold_out[:, i:i+1] = ... # back to you
        Phi_pred[:, i:i+1] = ... # back to you
        
    # 2. solve the linear system
    w = ... # back to you

    # 3. make predictions at training and test points
    f = ... # back to you
    f_hold_out = ... # back to you
    f_pred = ... # back to you
    
    # 4. compute the error and append it to a list.
    error = ... # back to you   
    error_list.append(error)
    
    # 5. plot the predictions
    fig1.add_subplot(max_order+1,2,index)
    plt.plot(x_pred, f_pred)
    plt.plot(x, y, 'rx')
    plt.ylim((2.5, 5.5))
    if (order ==0):
        plt.title('Predictions for Order ' + str(order) + ' model.')
    
    
    fig1.add_subplot(max_order+1,2,index+1)
    plt.plot(np.arange(0, order+1), np.asarray(error_list))
    plt.xlim((0, order+1))
    plt.ylim((0, np.max(error_list)))
    if (order ==0):
        plt.title('Training Error')
    index= index+2

plt.show()
#display(fig)
print('Holdout error list: ', error_list)

**Discussion:** What is going on here? Does this match your earlier findings, or your intuition about which model order was most appropriate? Why isn't held-out error behaving the same as training error?

## Regularising the model, using ridge regression

A nice way to limit model complexity is *regularisation* where model parameters are penalised from moving to silly values. Here we consider silly as high magnitude, which means the model is getting overly confident. Can you explain why this might be a problem? 

For this exercise, we'll use a 6th order model, which you might consider much too powerful for this simple problem. As a first step, we'll preprocess the features to ensure they are all operating in a similar range. E.g., $2000^6 >> 2000^1$, which means the weights for the 6th order features will take on radically different values to the 1st order features. To correct for this, and allow regularisation with a single constant, we'll normalize (z-score) the columns of training Phi to have zero mean and unit standard deviation. This same transformation is also applied to the testing basis matrices.

In [ ]:
order = 6
Phi = np.zeros((x_train.shape[0], order+1))
Phi_pred = np.zeros((num_pred_data, order+1))
Phi_hold_out = np.zeros((x_hold_out.shape[0], order+1))
for i in range(0, order+1):
    Phi[:, i:i+1] = x_train**i
    if i > 0:
        mean = Phi[:, i:i+1].mean()
        std = Phi[:, i:i+1].std()
        print(i,mean,std)
    else: # as the first column is constant, need to avoid divide by zero 
        mean = 0
        std = 1
    
    Phi[:, i:i+1] = (Phi[:, i:i+1] - mean) / std
    Phi_hold_out[:, i:i+1] = (x_hold_out**i - mean) / std
    Phi_pred[:, i:i+1] = (x_pred**i - mean) / std


In [ ]:
#Next we'll perform training, trying out different values of the regularisation coefficient, lambda.
error_list = []
train_error_list = []
lambdas = [1e-10, 1e-6, 1e-4, 1e-2, 1, 100] 
order = 6
#fig, axes = plt.subplots(nrows=1, ncols=3)
fig1=plt.figure(figsize=(16,order*4))
index =1
for l, lamba in enumerate(lambdas):
    # 1. build the basis set using x_train, x_hold_out
    # done above
        
    # 2. solve the linear system
    w = np.linalg.solve(np.dot(Phi.T, Phi) + lamba * np.eye(order+1), np.dot(Phi.T, y_train))

    # 3. make predictions at training and test points
    f = np.dot(Phi, w)
    f_hold_out = np.dot(Phi_hold_out, w)
    f_pred = np.dot(Phi_pred, w)
    
    # 4. compute the error and append it to a list.
    error = ((y_hold_out-f_hold_out)**2).sum()    
    error_list.append(error)
    train_error = ((y_train-f)**2).sum()    
    train_error_list.append(train_error)
    
    # 5. plot the predictions
    fig1.add_subplot(len(lambdas)+1,3,index)
    plt.plot(x_pred, f_pred)
    plt.plot(x, y, 'rx')
    plt.ylim(2.5, 5.5)
    if (l==0):
        plt.title('Pred. for Lambda ' + str(lamba))
    else: 
        plt.title(str(lamba))
        
    fig1.add_subplot(len(lambdas)+1,3,index+1)
    plt.plot(lambdas[:l+1], np.asarray(error_list))
    plt.xlim((min(lambdas), max(lambdas)))
    plt.xscale('log')
    plt.ylim(0, 12)
    if (l==0):
        plt.title('Held-out Error (validation/testing)')
    
    
    fig1.add_subplot(len(lambdas)+1,3,index+2)
    plt.plot(lambdas[:l+1], np.asarray(train_error_list))
    plt.xlim(min(lambdas), max(lambdas))
    plt.xscale('log')
    plt.ylim(0, 12)
    if (l == 0):
        plt.title('Training Error')
    index= index+3

plt.show()
#display(fig)
print('Holdout error list: ',error_list)

**Discussion:** What setting gives the best heldout performance? How does this relate to the training error, and can you describe whether you see evidence of overfitting or underfitting?